<a href="https://colab.research.google.com/github/mansi35/C2CL-Fingerprint-Matching/blob/main/preprocess_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import cv2
from PIL import Image, ImageOps, ImageEnhance
import numpy as np
import os
import random
from skimage import util, filters, morphology
from os.path import basename, join
import numpy as np
from glob import glob

In [4]:
contact_images_second_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session'
processed_contact_images_second_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/processed_contact-based_fingerprints/second_session'
contact_images_first_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session'
processed_contact_images_first_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/processed_contact-based_fingerprints/first_session'
processed_contactless_images_first_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contactless-npy/first_session'
processed_contactless_images_second_session = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contactless-npy/second_session'

In [5]:
# global variables
EXPAND_WIDTH = 220
EXPAND_HEIGHT = 220

CropWidth = 220
CropHeight = 220
ExpWidth = 220
ExpHeight = 220

# get fingerprint region for crop
def get_fp_region(image, crop_width=220, crop_height=220):
    CropWidth = crop_width
    CropHeight = crop_height
    ExpWidth = EXPAND_WIDTH
    ExpHeight = EXPAND_HEIGHT

    #otsu thresholding
    thresh = filters.threshold_otsu(image)

    #morphological closing
    picBW = image > thresh
    bw = morphology.closing(image > thresh, morphology.square(3))

    cleared = bw.copy()
    img_width = image.shape[1]
    img_height = image.shape[0]

    crop_l = img_width
    crop_r = 0
    crop_t = img_height
    crop_b = 0
    for i in range(img_height):
        for j in range(img_width):
            if cleared[i, j] == False:
                if (crop_l > j):
                    crop_l = j
                if (crop_r < j):
                    crop_r = j
                if (crop_t > i):
                    crop_t = i
                if (crop_b < i):
                    crop_b = i

    if ((crop_r - crop_l) < CropWidth):
        diff = CropWidth - (crop_r - crop_l)
        if (crop_r + crop_l > CropWidth): # right
            if (img_width - crop_r > diff / 2):
                crop_r += diff / 2
                crop_l -= diff / 2
            else:
                crop_r = img_width - 1
                crop_l = crop_r - (CropWidth + 2)
        else: # left
            if (crop_l > diff / 2):
                crop_l -= diff / 2
                crop_r += diff / 2
            else:
                crop_l = 1
                crop_r = crop_l + (CropWidth + 2)
    if ((crop_b - crop_t) < CropHeight):
        diff = CropHeight - (crop_b - crop_t)
        if (crop_b + crop_t > CropHeight): # bottom
            if (img_height - crop_b > diff / 2):
                crop_b += diff / 2
                crop_t -= diff / 2
            else:
                crop_b = img_height - 1
                crop_t = crop_b - (CropHeight + 2)
        else: # top
            if (crop_t > diff / 2):
                crop_t -= diff / 2
                crop_b += diff / 2
            else:
                crop_t = 1
                crop_b = crop_t + (CropHeight + 2)

    # expand region for rotation
    crop_l = (crop_r + crop_l - CropWidth) / 2
    crop_r = crop_l + CropWidth
    crop_t = (crop_t + crop_b - CropHeight) / 2
    crop_b = crop_t + CropHeight
    crop_l = (int)(crop_l - ((ExpWidth - CropWidth) / 2))
    crop_r = (int)(crop_r + ((ExpWidth - CropWidth) / 2))
    crop_t = (int)(crop_t - ((ExpHeight - CropHeight) / 2))
    crop_b = (int)(crop_b + ((ExpHeight - CropHeight) / 2))

    # check expanded region
    diff = 0
    if (crop_l < 0):
        diff = 0 - crop_l
        crop_l = crop_l + diff
        crop_r = crop_r + diff
    if (crop_r >= img_width):
        diff = crop_r - (img_width - 1)
        crop_l = crop_l - diff
        crop_r = crop_r - diff

    diff = 0
    if (crop_t < 0):
        diff = 0 - crop_t
        crop_t = crop_t + diff
        crop_b = crop_b + diff
    if (crop_b >= img_height):
        diff = crop_b - (img_height - 1)
        crop_t = crop_t - diff
        crop_b = crop_b - diff

    return (crop_l, crop_t, crop_r, crop_b)

def cropToRegionInterest(image_arr, img_fileName, save_url = ''):
    (crop_l, crop_t, crop_r, crop_b) = get_fp_region(image_arr)
    # crop for process image
    crop_x = (ExpWidth - CropWidth) / 2
    crop_y = (ExpHeight - CropHeight) / 2
    img = Image.fromarray(image_arr, 'L')
    img = img.crop([crop_l, crop_t, crop_r, crop_b])

    # single crop - save
    img_c = img.crop([crop_x, crop_y, crop_x + CropWidth, crop_y + CropHeight])
    img_path_new = join(save_url, img_fileName)
    img_c.save(img_path_new)

In [6]:
def createPersonFingerprintsDirectories(n, baseDir):
  for i in range(1, n + 1):
    os.mkdir(baseDir + '/p' + str(i) + '/')

In [ ]:
createPersonFingerprintsDirectories(336, contact_images_first_session)

In [ ]:
all_images = glob(join(contact_images_first_session, '*.jpg'))

for j, img_path in enumerate(all_images):
  img = os.path.basename(img_path)
  img_folder = 'p' + img.split('_')[0]
  os.rename(contact_images_first_session + '/' + img, contact_images_first_session + '/' + img_folder + '/' + img)


In [ ]:
createPersonFingerprintsDirectories(336, processed_contact_images_first_session)

In [ ]:
all_images = glob(join(contact_images_first_session, '**/*.jpg'))
for j, img_path in enumerate(all_images):
  print(img_path + ",   "+ str(j))
  img_fileName = os.path.basename(img_path)
  img_folder = 'p' + img_fileName.split('_')[0]
  image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  cropToRegionInterest(image, img_fileName, processed_contact_images_first_session + '/' + img_folder + '/')


/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_2.jpg,   0
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_4.jpg,   1
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_3.jpg,   2
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_5.jpg,   3
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_6.jpg,   4
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p1/1_1.jpg,   5
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p2/2_2.jpg,   6
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/first_session/p2/2_3.jpg,   7
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/firs

In [ ]:
createPersonFingerprintsDirectories(240, contact_images_second_session)

In [ ]:
all_images = glob(join(contact_images_second_session, '*.jpg'))

for j, img_path in enumerate(all_images):
  img = os.path.basename(img_path)
  img_folder = 'p' + img.split('_')[0]
  os.rename(contact_images_second_session + '/' + img, contact_images_second_session + '/' + img_folder + '/' + img)


In [ ]:
createPersonFingerprintsDirectories(240, processed_contact_images_second_session)

In [ ]:
all_images = glob(join(contact_images_second_session, '**/*.jpg'))
for j, img_path in enumerate(all_images):
  print(img_path + ",   " + str(j))
  img_fileName = os.path.basename(img_path)
  img_folder = 'p' + img_fileName.split('_')[0]
  image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  cropToRegionInterest(image, img_fileName, processed_contact_images_second_session + '/' + img_folder + '/')

/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_2.jpg,   0
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_1.jpg,   1
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_4.jpg,   2
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_5.jpg,   3
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_6.jpg,   4
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p1/1_3.jpg,   5
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p2/2_2.jpg,   6
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerprints/second_session/p2/2_5.jpg,   7
/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based_fingerpri

In [ ]:
img_data_first = np.array(glob(join(processed_contact_images_first_session, '**/*.jpg')))
img_data_second = np.array(glob(join(processed_contact_images_second_session, '**/*.jpg')))
print(len(img_data_first) + len(img_data_second))

2976


In [ ]:
save_url = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based-npy/first_session/'
img_data_first = glob(join(processed_contact_images_first_session, '**/*.jpg'))
for j, img in enumerate(img_data_first):
  img_fileName = basename(img).split('.')[0]
  img = cv2.imread(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  npy_image = np.array(img)
  img_fileName =  img_fileName +'.npy'
  np.save(join(save_url, img_fileName), npy_image)
    

In [ ]:
save_url = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contact-based-npy/second_session/'
img_data_first = glob(join(processed_contact_images_second_session, '**/*.jpg'))
for j, img in enumerate(img_data_first):
  img_fileName = basename(img).split('.')[0]
  img = cv2.imread(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  npy_image = np.array(img)
  img_fileName =  img_fileName +'.npy'
  np.save(join(save_url, img_fileName), npy_image)
    

In [ ]:
save_url = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contactless-npy/first_session/'
img_data_first = glob(join(processed_contact_images_second_session, '**/*.jpg'))
for j, img in enumerate(img_data_first):
  img_fileName = basename(img).split('.')[0]
  img = cv2.imread(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  npy_image = np.array(img)
  img_fileName =  img_fileName +'.npy'
  np.save(join(save_url, img_fileName), npy_image)

In [ ]:
save_url = '/content/drive/MyDrive/Cross_Fingerprint_Images_Database/contactless-npy/second_session/'
img_data_first = glob(join(processed_contact_images_second_session, '**/*.jpg'))
for j, img in enumerate(img_data_first):
  img_fileName = basename(img).split('.')[0]
  img = cv2.imread(img)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  npy_image = np.array(img)
  img_fileName =  img_fileName +'.npy'
  np.save(join(save_url, img_fileName), npy_image)